# Assignment: Simulate a Negotiation Scenario Between Two Planning Agents


---

### Objective:
This assignment challenges you to design and implement a **negotiation scenario between two AI planning agents** using a multi-agent framework. Each agent will have its own goals, preferences, and a strategy for negotiation. The goal is to observe how these agents interact, propose solutions, and potentially reach a mutually agreeable outcome or a stalemate. This will demonstrate the complexities of multi-agent communication and strategic decision-making in a constrained environment.

---

### Instructions:
1.  **LLM Access**: You'll need access to an LLM API (e.g., OpenAI's GPT-4o, GPT-4, GPT-3.5-turbo). Configure your API key securely.
2.  **Multi-Agent Framework**: **AutoGen** is highly recommended for simulating the agent interactions. Its flexibility in defining roles and conversational patterns makes it ideal for this task.
3.  **Scenario: Project Task Allocation Negotiation**: Imagine two project managers (Agent A and Agent B) who need to allocate a set of tasks for an upcoming project. They have different preferences for tasks and deadlines.
    * **Agent A's Goal**: Prioritize Task X and complete it by Monday.
    * **Agent B's Goal**: Prioritize Task Y and ensure no task extends beyond Wednesday.
    * **Tasks**: A list of tasks with estimated efforts and ideal deadlines (e.g., `{'Task A': {'effort': 3, 'deadline': 'Tuesday'}, ...}`).
    * **Constraints**: Total effort per agent per day, or combined effort constraints.
4.  **Agent Design**: Create at least two `AssistantAgent`s. Each agent should have:
    * A distinct **role** and **system message** defining its personality and negotiation objectives.
    * The ability to propose a task allocation plan.
    * The ability to evaluate a proposal from the other agent against its own goals.
    * The ability to counter-propose or accept/reject proposals.
    * A mechanism to indicate successful negotiation or a stalemate (e.g., a specific termination message).
5.  **User Proxy**: A `UserProxyAgent` will act as an impartial observer or a facilitator, initiating the negotiation and signaling termination.
6.  **Jupyter Notebook**: All your code, outputs, observations, and analysis must be documented in this Jupyter Notebook.
7.  **Analysis**: Explain your agent design, negotiation strategy, and the observed outcomes. Discuss the challenges and insights gained from simulating this scenario.

---

## Part 1: Setup and LLM Configuration
Configure your LLM and install necessary libraries.

### Task 1.1: Install Libraries and Configure LLM
Install `pyautogen` and `python-dotenv`. Set up your LLM configuration.

In [ ]:
# Install necessary libraries (if not already installed)
# !pip install pyautogen python-dotenv --quiet

import autogen
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# --- IMPORTANT: Create a .env file in the same directory as this notebook with the line: ---
# OPENAI_API_KEY="YOUR_OPENAI_API_KEY_HERE"

# Configure LLM (OpenAI model recommended)
llm_config = {
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={
            "model": ["gpt-4o", "gpt-4", "gpt-3.5-turbo"], # Use capable models for negotiation
        },
    ),
    "temperature": 0.7, # Higher temperature for more creative negotiation responses
}

# Fallback if OAI_CONFIG_LIST not found or for direct API key usage
if not llm_config["config_list"] and os.getenv("OPENAI_API_KEY"):
    print("Using OPENAI_API_KEY from environment variable as fallback for LLM config.")
    llm_config["config_list"] = [
        {"model": os.getenv("OPENAI_MODEL_ID", "gpt-4o"), "api_key": os.getenv("OPENAI_API_KEY")}
    ]
elif not llm_config["config_list"]:
    print("WARNING: No LLM configuration found. Please set OPENAI_API_KEY or create OAI_CONFIG_LIST.")

print("AutoGen environment and LLM configured!")

### Task 1.2: Define Project Tasks and Deadlines
Create a dictionary representing the tasks, their estimated effort (e.g., in person-hours or points), and preferred deadlines. Also define the allowed negotiation parameters (e.g., available days).

In [ ]:
project_tasks = {
    "Task A": {"effort": 5, "preferred_deadline": "Monday"},
    "Task B": {"effort": 3, "preferred_deadline": "Tuesday"},
    "Task C": {"effort": 7, "preferred_deadline": "Wednesday"},
    "Task D": {"effort": 4, "preferred_deadline": "Tuesday"},
    "Task E": {"effort": 6, "preferred_deadline": "Thursday"}
}

available_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
daily_effort_capacity = 10 # Max effort an agent can handle per day

print("Project tasks and constraints defined!")

---

## Part 2: Design Agents with Negotiation Strategies
Create two `AssistantAgent`s, each with distinct goals and a system message that guides its negotiation behavior. You'll also need a `UserProxyAgent` to facilitate.

### Task 2.1: `UserProxyAgent` (The Facilitator)
This agent will initiate the negotiation and serve as the human interface for observing the conversation. It should be able to terminate the chat.

In [ ]:
def is_termination_message(message):
    content = message.get("content", "").upper()
    # Agents should explicitly state 'AGREEMENT REACHED' or 'NO AGREEMENT' for termination
    return "AGREEMENT REACHED" in content or "NO AGREEMENT" in content or "TERMINATE" in content

user_proxy = autogen.UserProxyAgent(
    name="Mediator",
    human_input_mode="NEVER", # Set to NEVER for full automation
    max_consecutive_auto_reply=20, # Allow ample turns for negotiation
    is_termination_msg=is_termination_message,
    system_message=(
        "You are an impartial mediator overseeing a task allocation negotiation. "
        "Your role is to initiate the negotiation, observe the proposals, and allow the agents to reach a conclusion. "
        "When either 'AGREEMENT REACHED' or 'NO AGREEMENT' is stated by the agents, conclude the chat with 'TERMINATE'."
    )
)

print("UserProxyAgent 'Mediator' created!")

### Task 2.2: `AgentA` (Project Manager Alpha)
Define `AgentA` with a system message reflecting its specific goals and negotiation approach.

In [ ]:
agent_a = autogen.AssistantAgent(
    name="Project_Manager_Alpha",
    system_message=(
        "You are Project Manager Alpha, a negotiator focused on efficiently allocating tasks for a project. "
        "Your primary goal is to ensure 'Task A' is completed by Monday and 'Task B' by Tuesday. "
        "You prefer to take on tasks with higher effort ratings if they align with your deadlines. "
        "When proposing a plan, list tasks assigned to you and their proposed deadlines. "
        "You must consider the daily_effort_capacity of 10 points per agent. "
        "If a satisfactory agreement is reached, state 'AGREEMENT REACHED'. If negotiations reach a deadlock, state 'NO AGREEMENT'."
    ),
    llm_config=llm_config,
)

print("Agent A 'Project_Manager_Alpha' created!")

### Task 2.3: `AgentB` (Project Manager Beta)
Define `AgentB` with a system message reflecting its specific goals and negotiation approach.

In [ ]:
agent_b = autogen.AssistantAgent(
    name="Project_Manager_Beta",
    system_message=(
        "You are Project Manager Beta, a negotiator focused on ensuring smooth project execution and preventing delays. "
        "Your primary goal is to ensure 'Task C' is completed by Wednesday and no task extends beyond Thursday. "
        "You prefer tasks with lower effort ratings if possible, but are flexible for critical tasks. "
        "When proposing a plan, list tasks assigned to you and their proposed deadlines. "
        "You must consider the daily_effort_capacity of 10 points per agent. "
        "If a satisfactory agreement is reached, state 'AGREEMENT REACHED'. If negotiations reach a deadlock, state 'NO AGREEMENT'."
    ),
    llm_config=llm_config,
)

print("Agent B 'Project_Manager_Beta' created!")

---

## Part 3: Orchestrate the Negotiation
Set up the `GroupChat` and `GroupChatManager` to allow the agents to communicate and negotiate.

### Task 3.1: Create `GroupChat` and `GroupChatManager`
Assemble your agents into a `GroupChat` and manage their conversation. Ensure the agents can freely speak to each other.

In [ ]:
agents = [user_proxy, agent_a, agent_b]

groupchat = autogen.GroupChat(
    agents=agents,
    messages=[],
    max_round=30, # Allow sufficient turns for negotiation
    speaker_selection_method="auto", # AutoGen decides who speaks next
    allow_repeat_speaker=False, # Encourage agents to take turns
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
)

print("GroupChat and GroupChatManager created!")

### Task 3.2: Initiate the Negotiation
Start the chat with an initial message from the `Mediator` that sets the stage for the negotiation.

In [ ]:
initial_negotiation_message = (
    f"Mediator: Project Managers Alpha and Beta, we need to allocate the following tasks "
    f"for the upcoming project. The total daily effort capacity for each of you is {daily_effort_capacity} points. "
    f"Here are the tasks with their effort and preferred deadlines:\n\n"
    f"{project_tasks}\n\n"
    f"Please discuss and propose a task allocation plan that satisfies both your primary goals "
    f"and the overall project constraints. Once an agreement is reached, state 'AGREEMENT REACHED' "
    f"and summarize the final plan. If you cannot agree, state 'NO AGREEMENT'. "
    f"Start your discussion."
)

print("\n--- Initiating Task Allocation Negotiation ---")

chat_history = user_proxy.initiate_chat(
    manager, # The manager agent to start the group chat
    message=initial_negotiation_message,
    clear_history=True, # Clear previous chat history
    silent=False, # Keep False to observe conversation flow
)

print("\n--- Negotiation Conversation Ended ---")

---

## Part 4: Analysis and Reflection
Review the entire chat history in the console output. Analyze the negotiation process and the outcome.

### Task 4.1: Negotiation Process Analysis
* **Opening Proposals**: Describe the initial proposals from each agent. Did they immediately try to satisfy their own primary goals?
* **Counter-Proposals and Concessions**: How did the agents respond to each other's proposals? Did they offer counter-proposals? Did you observe any explicit or implicit concessions being made by either agent to reach an agreement?
* **Turn-Taking and Communication**: How well did the agents manage turn-taking and communication? Was the dialogue clear and constructive, or did it become repetitive or confused at any point?

### Task 4.2: Outcome Evaluation
* **Agreement Reached?**: Was a final agreement reached? If so, what was the agreed-upon task allocation plan and deadlines?
* **Goal Satisfaction**: To what extent did the final agreement (or lack thereof) satisfy the primary goals of `AgentA` and `AgentB`? Were some goals prioritized over others?
* **Stalemate Analysis**: If no agreement was reached, at what point did the negotiation break down? What were the key sticking points that prevented a resolution?

### Task 4.3: Agent Design and Limitations
* **System Message Impact**: How did your design of the system messages influence the agents' negotiation behavior and their ability to reach an agreement?
* **LLM Role**: What role did the underlying LLM play in enabling this negotiation? What aspects of its capabilities (e.g., understanding complex instructions, generating creative responses, maintaining context) were most critical?
* **Limitations**: What are the current limitations of this simulated negotiation? Consider:
    * Lack of explicit 'negotiation strategy' logic within agents (it's largely LLM-driven).
    * Difficulty in quantifying 'utility' or 'satisfaction' for each agent.
    * Scalability to many tasks or many agents.
    * Handling ambiguous or conflicting requirements.
* **Future Improvements**: How could you improve this negotiation simulation? Consider adding:
    * Tools for agents to evaluate proposed plans programmatically.
    * More sophisticated negotiation tactics (e.g., offering trade-offs, conditional proposals).
    * A formal utility function for each agent to guide their decisions.
    * A mechanism for the mediator to intervene or suggest compromises.

---

### Submission:
* Ensure all code cells have been executed and their outputs are visible.
* All analysis and reflections are clearly written in markdown cells.
* Make sure your `.env` file (or equivalent API key setup) is mentioned but **NOT** included in the submitted notebook for security reasons.
* Save your Jupyter Notebook as `[YourName]_Agent_Negotiation_Assignment.ipynb`.